<a href="https://colab.research.google.com/github/aalvarez359/3dunet-vs-medam-brain-tumor/blob/main/inference/brats2024/3d_unet_inf_brt2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run all scripts for unet brats2024 t2f and t2w inference

make sure correct modality is selected in Script #3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install nibabel segmentation-models-pytorch-3d torch torchvision torchaudio tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.8/626.8 kB 53.4 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16426 sha256=dbe8e77814972025953c8d7273d5ce4daf5990a12c93bd16af8c5c78e72870fc
  Stored in directory: /root/.cache/pip/wheels/9c/3f/43/e6271c7026fe08c185da2be23c98c8e87477d3db63f41f32ad
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=f72fd47f2e212ca6aa6e2a6c4fe823ba73aff06d1ee444507aae2a93078f83aa
  Stored in directory: /root/.cache/pip/wheels/4c/01/56/40a48f7

#3

In [ ]:
# --- Imports & basic config ---

import os, json, random, time
from pathlib import Path

import numpy as np
import nibabel as nib
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

import segmentation_models_pytorch_3d as smp3d

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

SEED = 42

# --------- MODALITY SWITCH ----------
# "t2f" (FLAIR-like) or "t2w"
MODALITY_NAME = "t2f"
# -----------------------------------

# BraTS 2024 paths
BRATS2024_ROOT = "/content/drive/MyDrive/BraTS2024_TrainingData_extracted/BraTS2024-BraTS-GLI-TrainingData/training_data1_v2"
JSON_PATH = os.path.join(BRATS2024_ROOT, "dataset_brats2024_cases.json")

# IMPORTANT: reuse the SAME split you used in training (lives in the T2F folder)
SPLIT_JSON_PATH = "/content/drive/MyDrive/BrainTumor_Checkpoints_BraTS2024_T2F/dataset_split_brats2024.json"

# Where your trained checkpoints live for this modality
SAVE_DIR = f"/content/drive/MyDrive/BrainTumor_Checkpoints_BraTS2024_{MODALITY_NAME.upper()}"

# 👉 Set this to the checkpoint you want to evaluate (best or last)
CHECKPOINT_PATH = os.path.join(
    SAVE_DIR,
     "best_epoch30_wholeDice0.7434.pt"
     #"best_epoch27_wholeDice0.6619.pt"
)

# Output folder for predicted NIfTIs & CSV
OUTPUT_ROOT = f"/content/drive/MyDrive/BraTS2024_Unet_Inference_{MODALITY_NAME.upper()}"
os.makedirs(OUTPUT_ROOT, exist_ok=True)

# Inference window config (should match training)
PATCH_SIZE = (128, 160, 160)
OVERLAP    = 0.5
N_CLASSES  = 4    # 0=bg, 1=ET, 2=NETC, 3=SNFH

def set_seed(s=SEED):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)

set_seed()
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)
print("Modality:", MODALITY_NAME)
print("Checkpoint:", CHECKPOINT_PATH)
print("Output dir:", OUTPUT_ROOT)


Device: cuda
Modality: t2f
Checkpoint: /content/drive/MyDrive/BrainTumor_Checkpoints_BraTS2024_T2F/best_epoch30_wholeDice0.7434.pt
Output dir: /content/drive/MyDrive/BraTS2024_Unet_Inference_T2F


In [ ]:
# --- NIfTI I/O & preprocessing ---

def percentile_clip(arr, lo=0.5, hi=99.5):
    a = arr.astype(np.float32)
    l, h = np.percentile(a, [lo, hi])
    a = np.clip(a, l, h)
    return a

def zscore_per_channel(x, eps=1e-8):
    # x: (C, D, H, W)
    x = x.astype(np.float32)
    for c in range(x.shape[0]):
        v = x[c]
        mask = (v != 0)
        if mask.any():
            m = v[mask].mean()
            s = v[mask].std()
        else:
            m = v.mean()
            s = v.std()
        s = max(float(s), eps)
        x[c] = (v - m) / s
    return x

def load_single_modality_chdw(path_img):
    """
    BraTS2024 modality loader.
    Input file is (H, W, D); output is (C=1, D, H, W).
    """
    img = nib.load(str(path_img))
    arr = img.get_fdata(dtype=np.float32)   # (H, W, D)
    assert arr.ndim == 3, f"{path_img} shape {arr.shape} not HWD"

    arr = percentile_clip(arr, 0.5, 99.5)
    vol = np.transpose(arr, (2, 0, 1))   # (D, H, W)
    vol = vol[None, ...]                 # (1, D, H, W)
    vol = zscore_per_channel(vol)
    return vol

def load_nifti_label_dhw(path_lab):
    """
    Label loader. Keeps only {0,1,2,3}.
    RC (4) and anything else are mapped to background (0).
    """
    lab = nib.load(str(path_lab)).get_fdata(dtype=np.float32)  # (H, W, D)
    lab = np.rint(lab).astype(np.int64)
    lab[lab < 0] = 0
    lab[lab > 3] = 0
    lab = np.transpose(lab, (2, 0, 1))   # (D, H, W)
    return lab

def save_pred_as_nifti(pred_dhw, ref_nii_path, out_path):
    """
    pred_dhw: (D,H,W) integer labels
    ref_nii_path: path to the original modality (for affine & header)
    """
    ref_img = nib.load(ref_nii_path)
    # Back to (H,W,D)
    arr_hwd = np.transpose(pred_dhw.astype(np.int16), (1, 2, 0))
    pred_nii = nib.Nifti1Image(arr_hwd, affine=ref_img.affine, header=ref_img.header)
    nib.save(pred_nii, out_path)

# --- Sliding-window inference (same as training) ---

@torch.no_grad()
def sliding_window_inference(volume_cdhw, model, window, overlap, device="cuda"):
    model.eval()
    C, D, H, W = volume_cdhw.shape
    wd, wh, ww = window
    sd = max(1, int(wd * (1 - overlap)))
    sh = max(1, int(wh * (1 - overlap)))
    sw = max(1, int(ww * (1 - overlap)))

    out_prob = torch.zeros((N_CLASSES, D, H, W), dtype=torch.float32, device=device)
    out_norm = torch.zeros((1, D, H, W), dtype=torch.float32, device=device)

    for z in range(0, max(D - wd + 1, 1), sd):
        z0 = min(z, D - wd)
        for y in range(0, max(H - wh + 1, 1), sh):
            y0 = min(y, H - wh)
            for x in range(0, max(W - ww + 1, 1), sw):
                x0 = min(x, W - ww)
                patch = volume_cdhw[:, z0:z0+wd, y0:y0+wh, x0:x0+ww]
                pt = torch.from_numpy(patch).unsqueeze(0).to(device)  # (1,C,D,H,W)
                logits = model(pt)                                    # (1,C,d,h,w)
                probs = F.softmax(logits, dim=1)[0]
                out_prob[:, z0:z0+wd, y0:y0+wh, x0:x0+ww] += probs
                out_norm[:, z0:z0+wd, y0:y0+wh, x0:x0+ww] += 1.0

    out_prob /= (out_norm + 1e-8)
    pred = torch.argmax(out_prob, dim=0)  # (D,H,W)
    return pred, out_prob

# --- Metrics: per class + whole tumor (label>0) ---

@torch.no_grad()
def dice_per_class(pred, target, n_classes=N_CLASSES):
    """
    pred, target: (D,H,W) int
    """
    if pred.ndim == 3:
        pred = pred.unsqueeze(0)
        target = target.unsqueeze(0)
    out = []
    for c in range(n_classes):
        p = (pred == c)
        t = (target == c)
        inter = (p & t).sum().item()
        denom = p.sum().item() + t.sum().item()
        out.append((2 * inter / denom) if denom > 0 else 1.0)
    return out

@torch.no_grad()
def dice_whole_tumor(pred, target, eps=1e-8):
    """
    Whole tumor = any label > 0 (ET, NETC, SNFH).
    """
    p = (pred > 0)
    t = (target > 0)
    inter = (p & t).sum().item()
    denom = p.sum().item() + t.sum().item()
    if denom == 0:
        return 1.0
    return 2.0 * inter / (denom + eps)


In [ ]:
# --- Load test split only ---

def load_test_items(split_json_path):
    assert os.path.exists(split_json_path), f"Split JSON not found: {split_json_path}"
    with open(split_json_path, "r") as f:
        split = json.load(f)
    test_items = split["test"]
    print(f"Loaded test set: {len(test_items)} cases")
    return test_items

test_items = load_test_items(SPLIT_JSON_PATH)

# --- Build model & load weights ---

def build_model(device):
    model = smp3d.Unet(
        encoder_name="efficientnet-b7",
        encoder_weights=None,
        in_channels=1,
        classes=N_CLASSES,
        decoder_channels=(192, 128, 64, 32, 16),
    )
    return model.to(device)

assert os.path.exists(CHECKPOINT_PATH), f"Checkpoint not found: {CHECKPOINT_PATH}"
ckpt = torch.load(CHECKPOINT_PATH, map_location="cpu")
model = build_model(device)
model.load_state_dict(ckpt["state_dict"])
model.to(device)
model.eval()

print("Loaded model from:", CHECKPOINT_PATH)


Loaded test set: 271 cases
Loaded model from: /content/drive/MyDrive/BrainTumor_Checkpoints_BraTS2024_T2F/best_epoch30_wholeDice0.7434.pt


In [ ]:
results = []

for it in tqdm(test_items, desc=f"U-Net {MODALITY_NAME} on BraTS2024 test set"):
    case_id = it["case_id"]
    img_path = it[MODALITY_NAME]
    lab_path = it["seg"]

    if img_path is None:
        print(f"Skipping {case_id}: no {MODALITY_NAME} path.")
        continue

    if not (os.path.exists(img_path) and os.path.exists(lab_path)):
        print(f"Missing files for {case_id}, skipping.")
        continue

    # 1) Load volume + label
    vol = load_single_modality_chdw(img_path)  # (1,D,H,W)
    lab = load_nifti_label_dhw(lab_path)       # (D,H,W)

    # 2) Inference
    pred, _ = sliding_window_inference(vol, model, PATCH_SIZE, OVERLAP, device=device)

    # 3) Dice metrics
    dices = dice_per_class(pred.cpu(), torch.from_numpy(lab))
    dice_bg, dice_ET, dice_NETC, dice_SNFH = dices
    dice_whole = dice_whole_tumor(pred.cpu(), torch.from_numpy(lab))

    print(
        f"{case_id}: "
        f"ET={dice_ET:.3f}  NETC={dice_NETC:.3f}  SNFH={dice_SNFH:.3f}  "
        f"Whole={dice_whole:.3f}"
    )

    # 4) Save prediction as NIfTI
    out_nii_path = os.path.join(
        OUTPUT_ROOT,
        f"{case_id}_UNet_{MODALITY_NAME}_seg.nii.gz"
    )
    save_pred_as_nifti(pred.cpu().numpy(), img_path, out_nii_path)

    # 5) Record row
    results.append({
        "case_id": case_id,
        "modality": MODALITY_NAME,
        "image_path": img_path,
        "label_path": lab_path,
        "pred_path": out_nii_path,
        "dice_bg": float(dice_bg),
        "dice_ET": float(dice_ET),
        "dice_NETC": float(dice_NETC),
        "dice_SNFH": float(dice_SNFH),
        "dice_whole_tumor": float(dice_whole),
    })

print("\n✅ Finished U-Net inference on test set.")
print("Total test cases processed:", len(results))

# Save CSV summary
if results:
    df = pd.DataFrame(results)
    csv_path = os.path.join(
        OUTPUT_ROOT,
        f"UNet_{MODALITY_NAME}_BraTS2024_test_results.csv"
    )
    df.to_csv(csv_path, index=False)
    print("Saved results CSV to:", csv_path)
else:
    print("No results to save.")


U-Net t2f on BraTS2024 test set:   0%|          | 1/271 [00:09<42:44,  9.50s/it]

BraTS-GLI-03064-100: ET=1.000  NETC=0.816  SNFH=0.000  Whole=0.826


U-Net t2f on BraTS2024 test set:   1%|          | 2/271 [00:13<28:44,  6.41s/it]

BraTS-GLI-02514-105: ET=1.000  NETC=0.827  SNFH=0.000  Whole=0.823
BraTS-GLI-02170-102: ET=0.000  NETC=0.875  SNFH=0.316  Whole=0.916


U-Net t2f on BraTS2024 test set:   1%|          | 3/271 [00:18<25:29,  5.71s/it]

...

BraTS-GLI-02995-101: ET=0.000  NETC=0.645  SNFH=0.140  Whole=0.621
BraTS-GLI-02082-106: ET=1.000  NETC=0.772  SNFH=1.000  Whole=0.772


U-Net t2f on BraTS2024 test set: 100%|█████████▉| 270/271 [19:54<00:04,  4.15s/it]

BraTS-GLI-02879-101: ET=1.000  NETC=0.837  SNFH=1.000  Whole=0.837


U-Net t2f on BraTS2024 test set: 100%|██████████| 271/271 [19:58<00:00,  4.42s/it]

BraTS-GLI-02996-101: ET=0.000  NETC=0.805  SNFH=0.010  Whole=0.762

✅ Finished U-Net inference on test set.
Total test cases processed: 271
Saved results CSV to: /content/drive/MyDrive/BraTS2024_Unet_Inference_T2F/UNet_t2f_BraTS2024_test_results.csv
